In [ ]:
#!pip install memory_profiler

In [1]:
%load_ext memory_profiler

In [2]:
import numpy as np

# ReLU

In [3]:
raw=np.random.randint(-10, 100, (100000, 400))/10

In [4]:
%timeit raw.clip(0, raw.max())

139 ms ± 2.32 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [5]:
%timeit raw[raw<0]==0

153 ms ± 1 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


# dict levels matters

In [6]:
from string import ascii_lowercase

In [7]:
asc={ascii_lowercase[i]:i for i in range(26)}
desc={ascii_lowercase[25-i]:i for i in range(26)}

In [8]:
asc_levels={'desc': desc, 'asc': asc}

In [9]:
%%timeit
for i in asc_levels:
    for j in asc_levels:
        j*2

475 ns ± 10.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


In [10]:
%timeit asc_levels['desc']['v']

48.7 ns ± 3.66 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


In [11]:
desc_levels={k: {'desc':desc[k], 'asc':asc[k]} for k in asc.keys()}

In [12]:
import pandas as pd

In [13]:
desc_levels=pd.DataFrame(asc_levels).T.to_dict()

In [14]:
%%timeit
for i in desc_levels:
    for j in desc_levels:
        j*2

46.7 µs ± 2.49 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


In [15]:
%timeit desc_levels['v']['asc']

49.3 ns ± 2.1 ns per loop (mean ± std. dev. of 7 runs, 10000000 loops each)


# replace 1d array values by dict

## generate an array

In [16]:
%memit
mn={
'янв' : '01',    
'фев' : '02',    
'мар' : '03',
'апр' : '04',
'май' : '05',
'июн' : '06',
'июл' : '07',
'авг' : '08',
'сен' : '09',
'окт' : '10',
'ноя' : '11',
'дек' : '12'
}

peak memory: 413.57 MiB, increment: 0.29 MiB


In [17]:
import itertools

In [18]:
ys=range(2010, 2020)
ms=mn.keys()
ds=range(1, 31)
d={'-'.join([str(y), m, str(d)]):'-'.join([str(y), mn[m], str(d)])
   for y, m, d in itertools.product(ys, ms, ds)
  }

In [19]:
%memit a=np.random.choice(list(d.keys()), int(5e7))

peak memory: 2846.59 MiB, increment: 2432.42 MiB


## as list

In [20]:
%memit b=list(a)

peak memory: 7962.72 MiB, increment: 5450.20 MiB


In [21]:
%memit b=a.tolist()

peak memory: 12949.51 MiB, increment: 4986.70 MiB


In [22]:
%time r=[d[k] for k in b]

CPU times: user 6.63 s, sys: 82.5 ms, total: 6.71 s
Wall time: 6.71 s


In [23]:
%memit r=[d[k] for k in b]

peak memory: 8328.89 MiB, increment: 381.46 MiB


In [24]:
%time r=[d[k] for k in a]

CPU times: user 21.3 s, sys: 104 ms, total: 21.4 s
Wall time: 21.4 s


In [25]:
%memit r=[d[k] for k in a]

peak memory: 8328.84 MiB, increment: 381.41 MiB


In [26]:
del r

In [27]:
del b

## vectorize

In [28]:
def replace(k, d): return d[k]
v_replace=np.vectorize(replace)

In [29]:
%time r=v_replace(a, d)

CPU times: user 20.3 s, sys: 3.61 s, total: 23.9 s
Wall time: 23.8 s


In [36]:
%memit r=v_replace(a, d)

peak memory: 12284.05 MiB, increment: 7270.24 MiB


In [30]:
del r

## unique reverse

In [31]:
%%time
val, inv = np.unique(a, return_inverse=True)
r = np.array([d[k] for k in val])[inv]

CPU times: user 53.3 s, sys: 3.64 s, total: 57 s
Wall time: 55.3 s


In [32]:
%%memit
val, inv = np.unique(a, return_inverse=True)
r = np.array([d[k] for k in val])[inv]

peak memory: 10211.58 MiB, increment: 5388.36 MiB


In [33]:
del r, val, inv

## fromiter

In [34]:
%time r=np.fromiter(map(lambda x: d[x], a), dtype=a.dtype)

CPU times: user 27.6 s, sys: 1.02 s, total: 28.6 s
Wall time: 28.6 s


In [35]:
%memit r=np.fromiter(map(lambda x: d[x], a), dtype=a.dtype)

peak memory: 7111.95 MiB, increment: 2097.89 MiB
